# 0 - Initializations

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import csv

from pylab import rcParams
import seaborn as sns
import math
import os
from datetime import datetime,  timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
 
from matplotlib import pyplot

from solarpy import irradiance_on_plane
from solarpy import solar_panel
from numpy import array

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Input, Dense, GRU, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, RepeatVector, Lambda, TimeDistributed, Embedding, TimeDistributed, BatchNormalization, Reshape, concatenate, Permute
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.backend import square, mean


from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from tensorflow.keras.utils import to_categorical

from keras.utils.vis_utils import plot_model

%run Database.ipynb
%run AuxFunctions.ipynb

plt.rcParams.update({'figure.figsize':(20,5), 'figure.dpi':300})

Collect from EDP database - True / Collect from local files - False

In [ ]:
df_pv, df_consumption = collectFromDatabase(False)

In [ ]:
dfinal = getData(local = True)

Data manipulation

In [ ]:
dataset1 = np.genfromtxt('dados__Rad.dat', delimiter=',', skip_header=3, missing_values='Missing',names=True,dtype=None)
dataset2 = np.genfromtxt('dados__Met.dat', delimiter=',', skip_header=3, missing_values='Missing',names=True,dtype=None)

df1 = pd.DataFrame(data=dataset1)
df2 = pd.DataFrame(data=dataset2)

df1.columns = ["Timestamp","RECORD","Avg_GHI","Avg_DHI","Avg_POA","Avg_DNI","Avg_cosDNI","Avg_closGHI","Std_GHI","Std_DHI","Std_POA","Std_DNI","Std_cosDNI","Std_closGHI","Min_GHI","Min_DHI","Min_POA","Min_DNI","Min_cosDNI","Min_closGHI","Max_GHI","Max_DHI","Max_POA","Max_DNI","Max_cosDNI","Max_closGHI","phi","theta","SensorT"]
df2.columns = ["Timestamp","RECORD","T_amb_min","T_amb_max","T_amb_avg","T_dp_avg","RH_min","RH_max","RH_avg","AH_min","AH_max","AH_avg","p_amb_min","p_amb_max","p_amb_avg","rho_act","v_min","v_max","v_avg","v_vectavg","v_dir_min","v_dir_max","v_dir_vectavg"]

df1['Timestamp'] = pd.to_datetime(df1['Timestamp'].str.decode("utf-8"), format='"%Y-%m-%d %H:%M:%S"')
df2['Timestamp'] = pd.to_datetime(df2['Timestamp'].str.decode("utf-8"), format='"%Y-%m-%d %H:%M:%S"')
df3 = pd.merge(df1,df2, how='inner', left_index=True, right_index=True)

Feature selection and data type selection

In [ ]:
PV = df_pv.copy()
consumption = df_consumption.copy()

consumption = consumption.set_index('Timestamp')
df3.rename(columns={"Timestamp_x": "Timestamp", "RECORD_x": "Record"}, inplace=True)

PV['Timestamp'] = PV['Timestamp'].dt.round('min')
PV = PV.groupby('Timestamp').mean().reset_index()
PV = PV.set_index('Timestamp')

idx = pd.period_range(PV.index[0], PV.index[-1], freq='min')
idy = pd.period_range(consumption.index[0], consumption.index[-1], freq='min')

PV = PV.reset_index()
PV = PV.set_index('Timestamp').resample("min").first().reset_index().reindex(columns=PV.columns)
cols = PV.columns.difference(['I1', 'I2', 'I3', 'V1', 'V2', 'V3', 'ActPwr'])
PV[cols] = PV[cols].ffill()
PV['ActPwr'] = PV['ActPwr']*1000

consumption = consumption.reset_index()
consumption = consumption.set_index('Timestamp').resample("min").first().reset_index().reindex(columns=consumption.columns)
cols = consumption.columns.difference(['Ir','Is','It','Vrs','Vst','Vtr','P','S'])
consumption[cols] = consumption[cols].ffill()

consumption = consumption.set_index('Timestamp')
df3 = df3.set_index('Timestamp')

PV_original = PV.copy()
PV = PV_original[['Timestamp', 'ActPwr']].copy()

consumption_original = consumption.copy()
consumption = consumption_original[['P']]

Sun irradiation theoretical computation

In [ ]:
panel = solar_panel(500, 0.15, id_name='EDP')  # surface, efficiency and name
panel.set_orientation(array([0, 0, -1]))  # upwards
panel.set_position(38.707089, -9.148882, 0)  # LISBON latitude, longitude, altitude

In [ ]:
PV = PV.reset_index()
PV['Theoretical Value'] = PV['Timestamp'].apply(lambda x: fnc(x.year, x.month, x.day, x.hour, x.minute))# year, month, day, hour, minute
PV = PV.set_index('Timestamp')

In [ ]:
PV['Theoretical Value'] = PV['Theoretical Value'].shift(60, axis = 0) #passar tudo para UTC
PV = PV.replace(np.nan, '-1')
#PV.to_csv("PV.csv")

# Merge meteo and PV datasets

In [ ]:
PREV = df4 = pd.merge(df3, PV, left_index=True, right_index=True)

In [ ]:
PREV = PREV.drop(columns=['Timestamp_y', 'RECORD_y'])

In [ ]:
PREV = PREV[['Avg_DHI', 'Avg_GHI', 'Avg_DNI', 'Avg_POA', \
             'T_amb_avg','RH_avg', 'AH_avg', 'p_amb_avg', 'rho_act', 'v_avg', 'v_dir_vectavg',    \
             'Theoretical Value', 'ActPwr']].copy()

In [ ]:
PREV = PREV.replace('-1', np.nan)

# Data interpolation


Adding 0's when there is no sun

In [ ]:
NOITE = PREV.copy()
NOITE['ActPwr_noite'] = NOITE.apply(f, axis=1)

In [ ]:
dataaux = NOITE.copy()

Interpolations for gaps of less than 30 minutes

In [ ]:
INT = NOITE.copy()
INT['Interpolation'] = INT['ActPwr_noite'].interpolate(method='polynomial', limit=30, order=2)

In [ ]:
dataaux['Interpolation'] = INT['Interpolation'].copy()

In [ ]:
INT['Interpolation'] = INT['Interpolation'].replace(np.nan, '-1000')
INT['Interpolation_Final'] = INT.apply(s, axis=1)
INT['Interpolation'] = INT['Interpolation'].replace('-1000',np.nan)

print(INT.ActPwr.count()/INT['Theoretical Value'].count() * 100)
print(INT.ActPwr_noite.count()/INT['Theoretical Value'].count() * 100)
print(INT.Interpolation.count()/INT['Theoretical Value'].count() * 100)
print(INT.Interpolation_Avg_GHI.count()/INT['Theoretical Value'].count() * 100)

# Dfinal -> Merge PV and Consumption datasets

In [ ]:
INT = INT.drop(['ActPwr', 'ActPwr_noite', 'Interpolation'], axis=1)
INT = INT.rename(columns={"Interpolation_Final": "ActPwr"})

In [ ]:
consumption = consumption.interpolate(method='polynomial', limit=30, order=2)

dfinal = pd.concat([INT, consumption], axis=1, sort = False, join = 'inner')
dfinal = dfinal.rename(columns={"Interpolation_Final": "ActPwr"})
dfinal = dfinal.reset_index()
dfinal = dfinal.set_index('Timestamp')

In [ ]:
dataaux['ActPwr_final'] = dfinal['ActPwr'].copy()

Data completion visualization

In [ ]:
plt.rcParams.update({'figure.figsize':(20,10), 'figure.dpi':300})
font = {'weight' : 'normal','size'   : 28}
plt.rc('font', **font)
pyplot.figure()

dataaux['ActPwr'].plot(label='Phase 5', linewidth=4, color='red')
dataaux['Theoretical Value'].plot(label='Phase 4', linewidth=1, color='blue',)

plt.title('Production Active Power')
plt.xlim('02/24/2020 06:30', '02/24/2020 19:00')
plt.ylim(-5,65000 )
plt.xlabel('Date')
plt.ylabel('[W]')
plt.legend()
axes = plt.gca()
plt.grid()
plt.savefig("int0")
pyplot.show()

# Build dataframes

In [ ]:
a1, a2, a3, a4, a5, v1, v2, v3, v4, t5, d6 = getDatasets(dfinal)

# PCA

In [ ]:
dt = a5.copy()
dataset = 'a5'

In [ ]:
X = dt.values # getting all values as a matrix of dataframe 
sc = StandardScaler() # creating a StandardScaler object
X_std = sc.fit_transform(X) # standardizing the data

In [ ]:
pca = PCA()
X_pca = pca.fit(X_std)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
num_components = 15
pca = PCA(num_components)  
X_pca = pca.fit_transform(X_std)

In [ ]:
pca = PCA(n_components = 0.99)
X_pca = pca.fit_transform(X_std) # this will fit and reduce dimensions
print(pca.n_components_) # one can print and see how many components are selected. In this case it is 4 same as above we saw in step 5

In [ ]:
pd.DataFrame(pca.components_, columns = dt.columns)

In [ ]:
n_pcs= pca.n_components_ # get number of component
# get the index of the most important feature on EACH component
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = dt.columns
# get the most important feature names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

In [ ]:
most_important_names.pop(2)

In [ ]:
most_important_names

In [ ]:
most_important_names.pop(5)

In [ ]:
dt = a5

In [ ]:
final_features = features()

In [ ]:
dt_selected = dt[final_features].copy()

In [ ]:
f = plt.figure(figsize=(30, 20))
ax = sns.heatmap(dt_selected.corr(), linewidths=3.5, annot=True, annot_kws={"size": 25}, fmt=".2f", cbar=True, cbar_kws = dict({"shrink": 1},use_gridspec=False,location="top"))
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90, fontsize = 30)
ax.set_yticklabels(ax.get_yticklabels(), rotation = 0, fontsize = 30)

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=30)


ax.figure.savefig("corr1.png", bbox_inches='tight')